In [30]:
import pandas as pd
import math

def Haversine(lat1, long1, lat2, long2):
    r = 6335.43e3

    lat1 = lat1*math.pi/180
    long1 = long1*math.pi/180

    lat2 = lat2*math.pi/180
    long2 = long2*math.pi/180

    t1 = math.sin(0.5*(lat2-lat1))*math.sin(0.5*(lat2-lat1))
    t2 = math.sin(0.5*(long2-long1))*math.sin(0.5*(long2-long1))

    sq = t1 + math.cos(lat1)*math.cos(lat2)*t2
    sq = max(0, sq)

    root = math.sqrt(sq)

    arcS = math.asin(root)

    dist = 2*r*arcS

    return dist

df_matches = pd.read_excel('All fixtures and Stadiums.xlsx')
df_loc = pd.read_excel('Stadiums X Y.xlsx')

loc_dict = {}
loc_dict["Team 1 Latitude"] = []
loc_dict["Team 1 Longitude"] = []
loc_dict["Team 2 Latitude"] = []
loc_dict["Team 2 Longitude"] = []
loc_dict["Distance"] = []

total_distance = 0

for i in range(len(df_matches)):

    team1 = df_matches.loc[i, "Team1"].strip()
    team2 = df_matches.loc[i, "Team2"].strip()

    team1Lat = None
    team1Long = None
    team2Lat = None
    team2Long = None
    
    for j in range(len(df_loc)):

        if df_loc.loc[j, "Country"].strip() == team1:
            team1Lat = df_loc.loc[j, "Latitude"] 
            team1Long = df_loc.loc[j, "Longitude"]

        if df_loc.loc[j, "Country"].strip() == team2:
            team2Lat = df_loc.loc[j, "Latitude"] 
            team2Long = df_loc.loc[j, "Longitude"]

    if team1Lat is None or team2Lat is None or team1Long is None or team2Long is None:
        print("Record Corrupt", team1Lat, team2Lat, team1Long, team2Long, team1, team2)
    else:
        loc_dict["Team 1 Latitude"].append(team1Lat)
        loc_dict["Team 1 Longitude"].append(team1Long)
        loc_dict["Team 2 Latitude"].append(team2Lat)
        loc_dict["Team 2 Longitude"].append(team2Long)

        dist = 2*Haversine(team1Lat, team1Long, team2Lat, team2Long)
        total_distance += dist

        loc_dict["Distance"].append(dist)

new_data_frame = pd.DataFrame(loc_dict)

df_matches = pd.concat((df_matches,new_data_frame),axis=1)

loc_dict = {}
loc_dict["Total Distance"] = []
for j in range(len(df_loc)):
    loc_dict["Total Distance"].append(0)

for i in range(len(df_matches)):

    team1 = df_matches.loc[i, "Team1"].strip()

    for j in range(len(df_loc)):
        if df_loc.loc[j, "Country"].strip() == team1:
            loc_dict["Total Distance"][j] += df_matches.loc[i, "Distance"]

new_data_frame = pd.DataFrame(loc_dict)
df_loc = pd.concat((df_loc,new_data_frame),axis=1)

total_distance = total_distance/1000

dupDict = {}
for i in range(len(df_loc)):

    nameO = df_loc.loc[i, "Country"].strip()
    
    for j in range(len(df_loc)):
        nameI = df_loc.loc[j, "Country"].strip()

        if nameI[-1].isdigit():
            if nameO == nameI[:-1]:
                if i not in dupDict.keys():
                    dupDict[i] = []
    
                dupDict[i].append(j)

dropList = []
for i in dupDict:
    for j in dupDict[i]:
        df_loc.loc[i, "Total Distance"] += df_loc.loc[j, "Total Distance"]
        dropList.append(j)

df_loc = df_loc.drop(dropList)

Groups = {}
# Ref: https://stackoverflow.com/questions/16476924/how-to-iterate-over-rows-in-a-pandas-dataframe
for j, row in df_loc.iterrows():
    grp = df_loc.loc[j, "Group"]

    if grp not in Groups:
        Groups[grp] = 0

    Groups[grp] += df_loc.loc[j, "Total Distance"]

print(f"Total Distance travelled is {total_distance:.2f} km.")

for grp in Groups:
    dist = Groups[grp]
    dist = dist / 1000

    L = []
    for j, row in df_loc.iterrows():
        g = df_loc.loc[j, "Group"]

        if g == grp:
            L.append(df_loc.loc[j, "Country"])
            
    print(f"Group {grp}, {L} travelled a distance of {dist:.2f} km.")

for j, row in df_loc.iterrows():
    dist = df_loc.loc[j, "Total Distance"]
    dist = dist / 1000

    name = df_loc.loc[j, "Country"]

    print(f"{name}'s team travelled a distance of {dist:.2f} km.")


Total Distance travelled is 1044783.45 km.
Group C, ['England', 'Italy', 'Malta', 'North Macedonia', 'Ukraine'] travelled a distance of 68967.51 km.
Group E, ['Albania', 'Czech Republic', 'Faroe Islands', 'Moldova', 'Poland'] travelled a distance of 82800.98 km.
Group I, ['Andorra', 'Belarus', 'Israel', 'Kosovo ', 'Romania', 'Switzerland'] travelled a distance of 121900.82 km.
Group D, ['Armenia', 'Croatia ', 'Latvia', 'Turkey', 'Wales'] travelled a distance of 106572.98 km.
Group F, ['Austria ', 'Azerbaijan', 'Belgium', 'Estonia', 'Sweden'] travelled a distance of 100084.07 km.
Group J, ['Bosnia and Herzegovia', 'Iceland', 'Liechtenstein', 'Luxembourg', 'Portugal', 'Slovakia'] travelled a distance of 126705.24 km.
Group G, ['Bulgaria', 'Hungary ', 'Lithuania', 'Montenegro', 'Serbia'] travelled a distance of 29741.00 km.
Group A, ['Cyprus', 'Georgia', 'Norway', 'Scotland', 'Spain'] travelled a distance of 139809.96 km.
Group H, ['Denmark', 'Finland', 'Khazakstan', 'Northern Ireland', '

In [31]:
df_matches

,Team1,Team2,Stadium,Team 1 Latitude,Team 1 Longitude,Team 2 Latitude,Team 2 Longitude,Distance
0,Bulgaria,Montenegro,Bulgaria,23.334513,42.688211,19.264492,42.444810,9.014732e+05
1,France,Netherlands,France,2.360128,48.924567,4.941859,52.314441,9.410633e+05
2,Gibraltar,Greece,Gibraltar,-5.350386,36.149367,23.741651,38.037227,6.446532e+06
3,Czech Republic,Poland,Czech Republic,14.415999,50.099877,21.044629,52.238634,1.533479e+06
4,Moldova,Faroe Islands,Moldova,28.868010,46.980292,-6.778280,62.019154,8.509365e+06
...,...,...,...,...,...,...,...,...
217,Croatia,Armenia,Croatia,16.017953,45.818681,44.525666,40.172001,6.392149e+06
218,Wales,Turkey,Wales,-3.182613,51.478255,28.765632,41.074458,7.402698e+06
219,Romania,Switzerland,Romania,26.152561,44.437284,7.464875,46.963200,4.166811e+06
220,Andorra,Israel,Andorra,1.517651,42.504878,34.824470,32.100682,7.671493e+06


In [26]:
for i in df_loc.row:
    print(i)

AttributeError: 'DataFrame' object has no attribute 'row'

In [32]:
df_loc

,Country,Group,Name of Stadium,Longitude,Latitude,Total Distance
0,England,C,Wembley Stadium,51.556325,-0.279025,2.028242e+07
1,Albania,E,Arena Kombëtare,41.318754,19.823824,1.419678e+07
2,Andorra,I,Estadi Nacional,42.504878,1.517651,2.536409e+07
3,Armenia,D,Vazgen Sargsyan Republican Stadium,40.172001,44.525666,2.609010e+07
4,Austria,F,Ernst Happel Stadium,48.207358,16.420953,1.561608e+07
5,Azerbaijan,F,Baku Olympic Stadium,40.430064,49.919590,3.207770e+07
6,Belarus,I,Dinamo Stadium,53.895368,27.560295,1.989136e+07
7,Belgium,F,King Baudouin Stadium,50.895852,4.334025,2.140238e+07
8,Bosnia and Herzegovia,J,Bilino Polje Stadium,44.205767,17.907130,1.179448e+07
9,Bulgaria,G,Vasil Levski National Stadium,42.688211,23.334513,5.491572e+06
